## Inicializando o modelo

In [ ]:
# OPCIONAL - API KEY é free para uso individual.
# https://smith.langchain.com/
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [1]:
from langchain_community.llms import Ollama 
llm = Ollama(base_url="http://ollama:11434", model="llama3", temperature = 0.1)

In [18]:
# llm.invoke("Quantos oscar o filme Oppenheimer ganhou em 2024")
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Escreva, em português, sobre {topic}")

# using LangChain Expressive Language chain syntax
chain = prompt | llm | StrOutputParser()


for chunk in chain.stream({"topic": "Quantos oscar o filme Oppenheimer ganhou em 2024"}):
    print(chunk, end="", flush=True)

Infelizmente, não há informações oficiais sobre o filme "Oppenheimer" ter ganho algum prêmio no Oscar em 2024. O filme foi lançado apenas em setembro de 2023 e a cerimônia do Oscar de 2024 ainda não ocorreu.

No entanto, é importante notar que o filme "Oppenheimer", dirigido por Christopher Nolan e estrelado por Cillian Murphy no papel titular, recebeu críticas positivas da imprensa especializada e do público. Embora não tenha sido lançado há muito tempo, é possível que o filme seja considerado para os prêmios do Oscar de 2024.

Para obter informações mais precisas sobre a performance do filme "Oppenheimer" nos prêmios do Oscar, é necessário aguardar a cerimônia oficial e as declarações oficiais da Academia de Artes e Ciências Cinematográficas.

## Indexing: Load

In [3]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path=("https://pt.wikipedia.org/wiki/Oppenheimer_(filme)"),
)
docs= loader.load()

In [4]:
docs[0].page_content

'\n\n\n\nOppenheimer (filme) – Wikipédia, a enciclopédia livre\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSaltar para o conteúdo\n\n\n\n\n\n\n\nMenu principal\n\n\n\n\n\nMenu principal\nmover para a barra lateral\nocultar\n\n\n\n\t\tNavegação\n\t\n\n\nPágina principalConteúdo destacadoEventos atuaisEsplanadaPágina aleatóriaPortaisInformar um erro\n\n\n\n\n\n\t\tColaboração\n\t\n\n\nBoas-vindasAjudaPágina de testesPortal comunitárioMudanças recentesManutençãoCriar páginaPáginas novasContatoDonativos\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBusca\n\n\n\n\n\n\n\n\n\n\n\nPesquisar\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCriar uma conta\n\nEntrar\n\n\n\n\n\n\n\n\nFerramentas pessoais\n\n\n\n\n\n Criar uma conta Entrar\n\n\n\n\n\n\t\tPáginas para editores sem sessão iniciada saber mais\n\n\n\nContribuiçõesDiscussão\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConteúdo\nmover para a barra lateral\nocultar\n\n\n\n\nInício\n\n\n\n\n\n1S

In [5]:
len(docs[0].page_content)

66231

## Indexing: Split

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [7]:
all_splits = text_splitter.split_documents(docs)

In [8]:
len(all_splits)

93

In [9]:
len(all_splits[1].page_content)

993

## Indexing: Store

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    base_url="http://ollama:11434",
    model="nomic-embed-text",
)

In [11]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [12]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [13]:
retriever.get_relevant_documents(
    "Quantos oscar o filme Oppenheimer ganhou em 2024?"
)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="2020 – presente\nGisaengchung (Parasita / Parasitas) – 2020\nNomadland (Nomadland / Nomadland - Sobreviver na América) – (2021)\nCODA (No Ritmo do Coração / CODA - No Ritmo do Coração) – (2022)\nEverything Everywhere All at Once (Tudo em Todo o Lado ao Mesmo Tempo /  \tTudo em Todo o Lugar ao Mesmo Tempo) – (2023)\nOppenheimer (2024)\nvdeBAFTA de Melhor Filme1948–1959\nThe Best Years of Our Lives (1948)\nHamlet (1949)\nLadri di biciclette (1950)\nAll About Eve (1951)\nLa ronde (1952)\nThe Sound Barrier (1953)\nJeux interdits (1954)\nLe Salaire de la peur (1955)\nRichard III (1956)\nGervaise (1957)\nThe Bridge on the River Kwai (1958)\n1960–1979\nRoom at the Top (1959)\nBen-Hur (1960)\nThe Apartment (1961)\nBallada o Soldate (1962)\nThe Hustler (1962)\nLawrence of Arabia (1963)\nTom Jones (1963)\nDr. Strangelove (1965)\nMy Fair Lady (1966)\nWho's Afraid of Virginia Woolf? (1967)\nA Man for All Seasons (1968)\nThe Graduate (1969)\nMidnight Cowboy (1970)\nButch Cas

## Recuperação e Geração:

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(""" Responda a pergunta com base apenas no contexto:
{context}
Pergunta: {input}
""")

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [17]:
# rag_chain.invoke("Quantos oscar o filme Oppenheimer ganhou em 2024?")

for chunk in rag_chain.stream({"input": "Quantos oscar o filme Oppenheimer ganhou em 2024"}):
    print(chunk, end="", flush=True)

De acordo com a informação fornecida, o filme "Oppenheimer" ganhou um Oscar em 2024.